In [438]:
import os
import time
import datetime
from collections import namedtuple
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tinvest

In [10]:
TOKEN = os.getenv("OPEN_API_TOKEN")

client = tinvest.SyncClient(TOKEN)

In [434]:
all_figi = client.get_market_stocks()

In [435]:
asset = namedtuple("Asset", ["name", "figi", "currency", "closed"])
ts_value = namedtuple("TsValue", ["dttm", "value"])

In [439]:
assets = list(map(lambda i: asset(i.name, i.figi, i.currency.name, []), (all_figi.payload.instruments)))
assets[:4]

[Asset(name='InterDigItal Inc', figi='BBG000HLJ7M4', currency='usd', closed=[]),
 Asset(name='RH', figi='BBG002293PJ4', currency='usd', closed=[]),
 Asset(name='Seagen Inc.', figi='BBG000BH0FR6', currency='usd', closed=[]),
 Asset(name='Navient', figi='BBG004MN1R41', currency='usd', closed=[])]

In [442]:
SLEEP_TIME_SECONDS = 60
DATE_FROM = datetime.datetime(2020, 11, 1)
DATE_TO = datetime.datetime(2021, 11, 1)

pbar = tqdm.tqdm(enumerate(assets), total=len(assets), leave=False)
pbar.set_description("Asset")
for i, a in pbar:
    a.closed.extend(
        list(map(
            lambda x: ts_value(x.time.date(), float(x.c)),
            client.get_market_candles(
                a.figi,
                from_=DATE_FROM,
                to=DATE_TO,
                interval=tinvest.schemas.CandleResolution.day
            ).payload.candles
        ))
    )
    
    if i % 90 == 0 and i >= 90:
        time.sleep(SLEEP_TIME_SECONDS)

In [443]:
# etf = assets
# share = assets

In [462]:
DAYS = 180

assets_df = pd.DataFrame(assets).dropna()
assets_df = assets_df[assets_df.closed.apply(len) > 1]
assets_df = assets_df.explode('closed')
assets_df = assets_df.reset_index(drop=True)

assets_df['dt'] = assets_df['closed'].apply(lambda x: x.dttm if not pd.isnull(x) else x)
assets_df['closed'] = assets_df['closed'].apply(lambda x: x.value if not pd.isnull(x) else x)
assets_df = assets_df[assets_df.dt >= (DATE_TO.date() - datetime.timedelta(days=DAYS))]

In [463]:
assets_df.groupby('figi').count()

,name,currency,closed,dt
figi,,,,
BBG000B9X8C0,125,125,125,125
BBG000B9XG87,125,125,125,125
BBG000B9XRY4,125,125,125,125
BBG000B9XYV2,125,125,125,125
BBG000B9YJ35,125,125,125,125
...,...,...,...,...
BBG011MCM288,77,77,77,77
BBG0122M8031,19,19,19,19
BBG0320C1036,125,125,125,125


In [464]:
unique_figi = assets_df.figi.drop_duplicates()
pbar = tqdm.tqdm(unique_figi, total=len(unique_figi), leave=False)
pbar.set_description("FIGI")
for figi in pbar:
    series_closed = assets_df[assets_df.figi == figi]['closed']
    
    assets_df.loc[assets_df.figi == figi, 'increase_rate'] = (
        series_closed / series_closed.shift()
    ).fillna(1)
    
    assets_df.loc[assets_df.figi == figi, 'total_increase'] = (
        assets_df.loc[assets_df.figi == figi, 'increase_rate']
        .cumprod()
        .tail(1)
        .reset_index(drop=True)
        [0]
    )

In [465]:
tmp = assets_df.groupby(["figi", "name", "currency"], as_index=False).agg(
    std_increase_rate=('increase_rate', 'std'),
    total_increase=('total_increase', 'max'),
    count=('dt', 'count'),
)

## 1) min std, max increase

In [466]:
(
    tmp[tmp.currency == 'rub']
    .sort_values('std_increase_rate')
    .head(100)
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
1167,BBG000RTHVK7,Группа Черкизово,rub,0.021042,1.567920,128
1415,BBG004S68CV8,ВСМПО-АВИСМА,rub,0.015902,1.489243,128
1408,BBG004S688G4,Акрон,rub,0.018584,1.478961,128
1368,BBG004730RP0,Газпром,rub,0.013105,1.477553,128
1396,BBG004S684M6,Газпром нефть,rub,0.013643,1.370049,128
1187,BBG000VFX6Y4,Globaltrans Investment PLC,rub,0.018167,1.333904,128
929,BBG000GQSVC2,Нижнекамскнефтехим - акции привилегированные,rub,0.020466,1.330357,128
1122,BBG000QJW156,Банк Санкт-Петербург,rub,0.016046,1.299714,128
1409,BBG004S689R0,ФосАгро,rub,0.018046,1.296759,128
1506,BBG007N0Z367,РусАгро,rub,0.016957,1.295075,128


In [467]:
(
    tmp[tmp.currency == 'usd']
    .sort_values('std_increase_rate')
    .head(100)
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
340,BBG000BLMY92,Mid-America Apartment Communities,usd,0.010480,1.336978,125
868,BBG000F6H8W8,Costco Wholesale,usd,0.010096,1.319570,125
132,BBG000BDJL83,FactSet Research Systems Inc,usd,0.010051,1.318942,125
5,BBG000B9Z0J8,Prologis,usd,0.010835,1.282718,125
799,BBG000D9D830,Accenture,usd,0.009908,1.239258,125
693,BBG000C41023,UDR,usd,0.010616,1.233726,125
442,BBG000BPXVJ6,Republic Services,usd,0.009948,1.230797,125
978,BBG000J6XT05,Exelon Corporation,usd,0.010232,1.221916,125
114,BBG000BCZL41,Verisk Analytics,usd,0.009805,1.218039,125
425,BBG000BP4MH0,Marsh & McLennan,usd,0.010240,1.211681,125


## 2) max std

In [468]:
(
    tmp[tmp.currency == 'rub']
    .sort_values('std_increase_rate', ascending=False)
    .head(100)
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
623,BBG000BX7DH0,ТНС энерго Воронеж,rub,0.070446,3.725664,128
1417,BBG004S68FR6,Мечел - Привилегированные акции,rub,0.040350,3.683230,128
1580,BBG00F6NKQX3,ГК Самолет,rub,0.028422,3.188937,128
1017,BBG000LWVHN8,Дагестанская энергосбытовая компания,rub,0.076831,2.728730,128
1034,BBG000N16BP3,ИСКЧ,rub,0.041348,2.306397,128
1182,BBG000V07CB8,ДЭК,rub,0.072757,2.216625,128
1160,BBG000RJWGC4,Ашинский метзавод,rub,0.048342,2.192424,128
1398,BBG004S68598,Мечел,rub,0.041793,1.861235,128
1195,BBG000W325F7,Русская аквакультура,rub,0.023754,1.853990,128
1118,BBG000QF1Q17,ДВМП,rub,0.025241,1.834592,128


In [469]:
(
    tmp[tmp.currency == 'usd']
    .sort_values('std_increase_rate', ascending=False)
    .head(100)
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
505,BBG000BS4MP5,Dillard's Inc,usd,0.042091,2.209540,125
1358,BBG003PHHZT1,Moderna Inc,usd,0.044623,2.119934,125
1696,BBG00QVJV6V4,Bill.com Holdings Inc,usd,0.040152,2.111717,125
1356,BBG003PDKJF7,Prothena Corporation,usd,0.050443,1.976786,107
1646,BBG00KT2SCV8,GreenSky Inc,usd,0.054127,1.883077,125
1505,BBG007KC7PB0,Intellia Therapeutics Inc,usd,0.061406,1.863509,125
1723,BBG00W0KZD98,Li Auto,usd,0.040166,1.733794,125
775,BBG000CZ4KF3,Stamps.com Inc,usd,0.065289,1.720571,101
1146,BBG000R1H6P8,Constellation Pharmaceuticals,usd,0.097283,1.626437,46
1544,BBG00B6F2F09,"SentinelOne, Inc.",usd,0.043283,1.562353,86


## 3) max insrease

In [470]:
(
    tmp[tmp.currency == 'rub']
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
623,BBG000BX7DH0,ТНС энерго Воронеж,rub,0.070446,3.725664,128
1417,BBG004S68FR6,Мечел - Привилегированные акции,rub,0.040350,3.683230,128
1580,BBG00F6NKQX3,ГК Самолет,rub,0.028422,3.188937,128
1017,BBG000LWVHN8,Дагестанская энергосбытовая компания,rub,0.076831,2.728730,128
1034,BBG000N16BP3,ИСКЧ,rub,0.041348,2.306397,128
1182,BBG000V07CB8,ДЭК,rub,0.072757,2.216625,128
1160,BBG000RJWGC4,Ашинский метзавод,rub,0.048342,2.192424,128
1398,BBG004S68598,Мечел,rub,0.041793,1.861235,128
1195,BBG000W325F7,Русская аквакультура,rub,0.023754,1.853990,128
1118,BBG000QF1Q17,ДВМП,rub,0.025241,1.834592,128


In [471]:
(
    tmp[tmp.currency == 'usd']
    .sort_values('total_increase', ascending=False)
    .head(10)
)

,figi,name,currency,std_increase_rate,total_increase,count
1277,BBG001WMKHH5,CloudFlare Inc,usd,0.031134,2.464186,125
1688,BBG00PPS73P4,Inmode Ltd,usd,0.039119,2.333785,125
1455,BBG005XZHR97,Veritiv Corp,usd,0.038562,2.239457,125
505,BBG000BS4MP5,Dillard's Inc,usd,0.042091,2.209540,125
1354,BBG003NJHZT9,Datadog Inc,usd,0.025863,2.176831,125
1358,BBG003PHHZT1,Moderna Inc,usd,0.044623,2.119934,125
1696,BBG00QVJV6V4,Bill.com Holdings Inc,usd,0.040152,2.111717,125
899,BBG000FVXD63,Range Resources,usd,0.037357,2.063717,125
1356,BBG003PDKJF7,Prothena Corporation,usd,0.050443,1.976786,107
1291,BBG0022FDRY8,MongoDB Inc,usd,0.034305,1.916578,125
